In [ ]:
# note: if you have already restored dependencies by looking at the requirements.txt file, you can skip this cell
# activating the environment is necessary to make sure the dependencies are installed in the right place
!source venv/bin/activate

# %pip runs pip directly from the notebook
%pip install -r requirements.txt

# Reading and Uploading files from Insta360

For my personal blogs I use an Insta360 x3 (as well as an Ace Pro) camera, although I have two different microsd cards I still struggle on keeping files stored somewhere I can reprocess later if needed.

So what if I created a script that does the hard lifting for me?

## Theory

Using Python I might be able to:

1. list all the files currently in the camera's SD Card
2. group them by date of creation and type (video x photos)
3. upload them to a cold storage on AWS / Azure
4. delete them from the camera in order to free up space

This notebook will test out if this could work!

## Listing all the files in my camera`s SD Card

Once mounted to an Unix system Insta360 cameras contain a `DCIM` folder which has subfolders containing actual footage recorded by that camera.

To make this easier I'll begin with the x3 files (a 360 camera). Looking at the file browser the path I'm interested in is `DCIM/Camera01`

Within those subfolders we'll find:

1. `*.lrv` files containing low resolution footage
2. `*.insv` files containing 360 footage in its native resolution
3. `*.insp` files containing 360 photos in its native resolution
4. `*.dng` files containing the digital negative of the photos

### Navigating to the Camera itself


In [6]:
from pathlib import Path

# make sure the data directory exists
# in Mac this means looking at the Volume mounted in /Volumes/<camera_name>
CAMERA_NAME = "Insta360 x3"

volumes_path = Path("/Volumes")
print(f"found volumes: {volumes_path}")
# check if there's a volume mounted with the camera name
camera_path = volumes_path / CAMERA_NAME
print(f"camera volume: {camera_path}")
if camera_path.exists():
    print("camera volume found")
else:
    exit(-1)

found volumes: /Volumes
camera volume: /Volumes/Insta360 x3
camera volume found


once we are in the camera's mounted volume we can move into the DCIM folder and check if there are any subdirs to it.

since I'm not sure if it'll always be "Camera01" I'll iterate and list each directory.

In [9]:
# moving into the DCIM folder
dcim_path = camera_path / "DCIM"
if dcim_path.exists():
    print(f"found DCIM folder: {dcim_path}")
else:
    exit(-2)

# list the contents of the DCIM folder
dcim_contents = list(dcim_path.iterdir())
# filter out files, keep only directories
dcim_contents = [item for item in dcim_contents if item.is_dir()]
print(f"DCIM subdirectories: {dcim_contents}")

found DCIM folder: /Volumes/Insta360 x3/DCIM
DCIM subdirectories: [PosixPath('/Volumes/Insta360 x3/DCIM/Camera01')]


### Iterating over files

Now that we have access to the Camera's content let's see what information we can extract for each files in there.

In [18]:
from datetime import datetime
files_path = dcim_contents[0] # assuming the first directory is the one we want
print(f"files path: {files_path}")

all_files = list(files_path.iterdir())
first_file = all_files[0]
print(f"first file: {first_file}, there are {len(all_files)} files in total")

file_metadata = first_file.stat()
print(f"file's size is {file_metadata.st_size} bytes, it was created on {datetime.fromtimestamp(file_metadata.st_ctime)}")

files path: /Volumes/Insta360 x3/DCIM/Camera01
first file: /Volumes/Insta360 x3/DCIM/Camera01/VID_20241119_071917_00_001.insv, there are 105 files in total
file's size is 632827501 bytes, it was created on 2024-11-19 07:20:41.010000


By using the `.stat()` method I'm able to fetch the metadata I expected to be able to consider uploading those files to AWS:

1. Its size
2. And its creation date

Now let's make this a bit more organized to make iterations easier in the future.

In [22]:
from enum import Enum
from typing import Self

class Insta360File:
    """
    A file from an Insta360 camera.
    """

    class Insta360FileType(Enum):
        """
        The type of file that an Insta360 camera can produce.
        """
        PHOTO = ".insp"
        VIDEO = ".insv"
        LOW_RES_VIDEO = ".lrv"
        PHOTO_NEGATIVE = ".dng"

    def __init__(self, path: Path) -> Self:
        """
        Initialize the Insta360File with the given path.
        :param path: the path to the file
        """
        self.__path__ = path
        self.__absolute__ = path.absolute()
        self.name = path.stem
        self.__file_type__ = path.suffix
        metadata = path.stat() # get the file metadata and copy it over to the parameters
        self.size = metadata.st_size
        self.created_at = datetime.fromtimestamp(metadata.st_ctime)

    @property
    def file_type(self) -> Insta360FileType:
        """
        Get the file type as an Insta360FileType enum.
        """
        for file_type in Insta360File.Insta360FileType:
            if self.__file_type__ == file_type.value:
                return file_type
        return None
    

# creating a File from our first_file
first_file_instance = Insta360File(first_file)
print(f"name: {first_file_instance.name} was created on {first_file_instance.created_at} and is of type {first_file_instance.file_type}")

name: VID_20241119_071917_00_001 was created on 2024-11-19 07:20:41.010000 and is of type Insta360FileType.VIDEO


now let's group out the files by their types

In [28]:
for file in all_files:
    file_instance = Insta360File(file)

all_files_objects = [Insta360File(file) for file in all_files]
print(f"created {len(all_files_objects)} file objects")

photo_files = [file for file in all_files_objects if file.file_type == Insta360File.Insta360FileType.PHOTO]
video_files = [file for file in all_files_objects if file.file_type == Insta360File.Insta360FileType.VIDEO]
low_res_video_files = [file for file in all_files_objects if file.file_type == Insta360File.Insta360FileType.LOW_RES_VIDEO]

created 105 file objects


### Preparing to upload a file to S3

Now that I'm able to iterate over files let's work on uploading it to S3 using `boto3`.

First, to make things secure, we'll load our `.env` file into the context as environment variables to ensure that the client is able to authenticate.

In [5]:
from dotenv import load_dotenv

if not load_dotenv():
    print("could not load .env file")
    exit(-3)
